# PyCity Schools Analysis

* Summary: From the following analysis, we can see some correlations between student performance (their maths and reading marks) and key factors from the school: budget per student, no. of students and type of school.
  
* Conclusions:
  * As evident from the Scores by Spending summary, a larger budget per student does not equate to better marks made by students.
  * As evident from the Scores by School Size summary, smaller school size is associated with higher marks with small to medium schools (with students of less than 2000) have an overall passing rate of 78-79%, whereas large schools (from 2000 and up to 5000 students) have this overall passing rate drop drastically to 70%.
---

In [29]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [3]:
school_data_complete

Student ID     student_name gender  year         school_name  \
0               0     Paul Bradley      M     9   Huang High School   
1               1     Victor Smith      M    12   Huang High School   
2               2  Kevin Rodriguez      M    12   Huang High School   
3               3    Richard Scott      M    12   Huang High School   
4               4       Bonnie Ray      F     9   Huang High School   
...           ...              ...    ...   ...                 ...   
39165       39165     Donna Howard      F    12  Thomas High School   
39166       39166        Dawn Bell      F    10  Thomas High School   
39167       39167   Rebecca Tanner      F     9  Thomas High School   
39168       39168     Desiree Kidd      F    10  Thomas High School   
39169       39169  Carolyn Jackson      F    11  Thomas High School   

       reading_score  maths_score  School ID         type  size   budget  
0                 96           94          0   Government  2917  1910635  
1                 90           43          0   Government  2917  1910635  
2                 41           76          0   Government  2917  1910635  
3                 89           86          0   Government  2917  1910635  
4                 87           69          0   Government  2917  1910635  
...              ...          ...        ...          ...   ...      ...  
39165             51           48         14  Independent  1635  1043130  
39166             81           89         14  Independent  1635  1043130  
39167             99           99         14  Independent  1635  1043130  
39168             72           77         14  Independent  1635  1043130  
39169             47           62         14  Independent  1635  1043130  

[39170 rows x 11 columns]

## Local Government Area Summary

In [4]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete["School ID"].unique())
student_count = len(school_data_complete["Student ID"])

# Calculate the Total Budget
total_budget = school_data["budget"].sum()

In [5]:
# Calculate the Average Scores
average_maths_score = school_data_complete["maths_score"].sum() / student_count
average_reading_score = school_data_complete["reading_score"].sum() / student_count

In [6]:
# Calculate the Percentage Pass Rates
passing_maths_count = school_data_complete[(school_data_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_maths_reading_count = school_data_complete[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)].count()["student_name"]

overall_passing_rate = passing_maths_reading_count / float(student_count) * 100

In [7]:
# Convert to DataFrame
area_summary = pd.DataFrame(
    {"Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Maths Score": [average_maths_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Maths": [passing_maths_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_rate]
    }
)

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary


Total Schools Total Students    Total Budget  Average Maths Score  \
0             15         39,170  $24,649,428.00            70.338192   

   Average Reading Score  % Passing Maths  % Passing Reading  \
0              69.980138        86.078632          84.426857   

   % Overall Passing  
0          72.808272

## School Summary

In [8]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count per school from school_data
per_school_counts = school_data.set_index(["school_name"])["size"]

# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data.set_index(["school_name"])["budget"]
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores per school from school_data_complete
per_school_maths = school_data_complete.groupby(["school_name"])["maths_score"].mean()
per_school_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()

In [9]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.
scm_filter = school_data_complete.loc[school_data_complete["maths_score"]>=50, :]
school_passing_maths = scm_filter.groupby(["school_name"]).count()["Student ID"]
spr_filter = school_data_complete.loc[school_data_complete["reading_score"]>=50, :]
school_passing_reading = spr_filter.groupby(["school_name"]).count()["Student ID"]

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.
pmr_filter = school_data_complete.loc[(school_data_complete["maths_score"]>=50) & (school_data_complete["reading_score"]>=50), :]
passing_maths_and_reading = pmr_filter.groupby(["school_name"]).count()["Student ID"]

In [10]:
#  Calculate the Percentage Pass Rates
per_school_passing_maths = school_passing_maths / per_school_counts * 100
per_school_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = passing_maths_and_reading / per_school_counts * 100

In [11]:
# Convert to DataFrame
per_school_summary = pd.DataFrame({"School Type": school_types,
                                   "Total Students": per_school_counts,
                                  "Total School Budget": per_school_budget,
                                  "Per Student Budget": per_school_capita,
                                  "Average Maths Score": per_school_maths, 
                                  "Average Reading Score": per_school_reading,
                                  "% Passing Maths": per_school_passing_maths,
                                   "% Passing Reading": per_school_passing_reading,
                                  "% Overall Passing": overall_passing_rate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

School Type  Total Students Total School Budget  \
school_name                                                              
Bailey High School      Government            4976       $3,124,928.00   
Cabrera High School    Independent            1858       $1,081,356.00   
Figueroa High School    Government            2949       $1,884,411.00   
Ford High School        Government            2739       $1,763,916.00   
Griffin High School    Independent            1468         $917,500.00   
Hernandez High School   Government            4635       $3,022,020.00   
Holden High School     Independent             427         $248,087.00   
Huang High School       Government            2917       $1,910,635.00   
Johnson High School     Government            4761       $3,094,650.00   
Pena High School       Independent             962         $585,858.00   
Rodriguez High School   Government            3999       $2,547,363.00   
Shelton High School    Independent            1761       $1,056,600.00   
Thomas High School     Independent            1635       $1,043,130.00   
Wilson High School     Independent            2283       $1,319,574.00   
Wright High School     Independent            1800       $1,049,400.00   

                       Per Student Budget  Average Maths Score  \
school_name                                                      
Bailey High School                  628.0            72.352894   
Cabrera High School                 582.0            71.657158   
Figueroa High School                639.0            68.698542   
Ford High School                    644.0            69.091274   
Griffin High School                 625.0            71.788147   
Hernandez High School               652.0            68.874865   
Holden High School                  581.0            72.583138   
Huang High School                   655.0            68.935207   
Johnson High School                 650.0            68.843100   
Pena High School                    609.0            72.088358   
Rodriguez High School               637.0            72.047762   
Shelton High School                 600.0            72.034072   
Thomas High School                  638.0            69.581651   
Wilson High School                  578.0            69.170828   
Wright High School                  583.0            72.047222   

                       Average Reading Score  % Passing Maths  \
school_name                                                     
Bailey High School                 71.008842        91.639871   
Cabrera High School                71.359526        90.850377   
Figueroa High School               69.077993        81.654798   
Ford High School                   69.572472        82.438846   
Griffin High School                71.245232        91.212534   
Hernandez High School              69.186408        80.949299   
Holden High School                 71.660422        89.929742   
Huang High School                  68.910525        81.693521   
Johnson High School                69.039277        82.062592   
Pena High School                   71.613306        91.683992   
Rodriguez High School              70.935984        90.797699   
Shelton High School                70.257808        91.538898   
Thomas High School                 69.768807        83.853211   
Wilson High School                 68.876916        82.785808   
Wright High School                 70.969444        91.777778   

                       % Passing Reading  % Overall Passing  
school_name                                                  
Bailey High School             87.379421          80.084405  
Cabrera High School            89.074273          80.785791  
Figueroa High School           82.807731          67.650051  
Ford High School               82.219788          67.469880  
Griffin High School            88.487738          81.335150  
Hernandez High School          81.877023          66.364617  
Holden High School             88.52459

## Top Performing Schools (By % Overall Passing)

In [12]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head(5)


School Type  Total Students Total School Budget  \
school_name                                                              
Griffin High School    Independent            1468         $917,500.00   
Cabrera High School    Independent            1858       $1,081,356.00   
Bailey High School      Government            4976       $3,124,928.00   
Wright High School     Independent            1800       $1,049,400.00   
Rodriguez High School   Government            3999       $2,547,363.00   

                       Per Student Budget  Average Maths Score  \
school_name                                                      
Griffin High School                 625.0            71.788147   
Cabrera High School                 582.0            71.657158   
Bailey High School                  628.0            72.352894   
Wright High School                  583.0            72.047222   
Rodriguez High School               637.0            72.047762   

                       Average Reading Score  % Passing Maths  \
school_name                                                     
Griffin High School                71.245232        91.212534   
Cabrera High School                71.359526        90.850377   
Bailey High School                 71.008842        91.639871   
Wright High School                 70.969444        91.777778   
Rodriguez High School              70.935984        90.797699   

                       % Passing Reading  % Overall Passing  
school_name                                                  
Griffin High School            88.487738          81.335150  
Cabrera High School            89.074273          80.785791  
Bailey High School             87.379421          80.084405  
Wright High School             86.666667          79.722222  
Rodriguez High School          87.396849          79.419855

## Bottom Performing Schools (By % Overall Passing)

In [13]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values("% Overall Passing")
bottom_schools.head(5)


School Type  Total Students Total School Budget  \
school_name                                                              
Hernandez High School   Government            4635       $3,022,020.00   
Huang High School       Government            2917       $1,910,635.00   
Johnson High School     Government            4761       $3,094,650.00   
Wilson High School     Independent            2283       $1,319,574.00   
Ford High School        Government            2739       $1,763,916.00   

                       Per Student Budget  Average Maths Score  \
school_name                                                      
Hernandez High School               652.0            68.874865   
Huang High School                   655.0            68.935207   
Johnson High School                 650.0            68.843100   
Wilson High School                  578.0            69.170828   
Ford High School                    644.0            69.091274   

                       Average Reading Score  % Passing Maths  \
school_name                                                     
Hernandez High School              69.186408        80.949299   
Huang High School                  68.910525        81.693521   
Johnson High School                69.039277        82.062592   
Wilson High School                 68.876916        82.785808   
Ford High School                   69.572472        82.438846   

                       % Passing Reading  % Overall Passing  
school_name                                                  
Hernandez High School          81.877023          66.364617  
Huang High School              81.453548          66.712376  
Johnson High School            81.978576          67.191766  
Wilson High School             81.296540          67.455103  
Ford High School               82.219788          67.469880

## Maths Scores by Year

In [14]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby(["school_name"])["maths_score"].mean()
year_ten_scores = year_ten.groupby(["school_name"])["maths_score"].mean()
year_eleven_scores = year_eleven.groupby(["school_name"])["maths_score"].mean()
year_twelve_scores = year_twelve.groupby(["school_name"])["maths_score"].mean()

maths_scores_by_year = pd.DataFrame({"Year 9": year_nine_scores,
                                     "Year 10": year_ten_scores,
                                     "Year 11": year_eleven_scores,
                                     "Year 12": year_twelve_scores})

# Minor data wrangling
maths_scores_by_year.index.name = None

# Display the DataFrame
maths_scores_by_year

Year 9    Year 10    Year 11    Year 12
Bailey High School     72.493827  71.897498  72.374900  72.675097
Cabrera High School    72.321970  72.437768  71.008299  70.604712
Figueroa High School   68.477804  68.331586  68.811001  69.325282
Ford High School       69.021609  69.387006  69.248862  68.617811
Griffin High School    72.789731  71.093596  71.692521  71.469178
Hernandez High School  68.586831  68.867156  69.154412  68.985075
Holden High School     70.543307  75.105263  71.640777  73.409639
Huang High School      69.081754  68.533246  69.431345  68.639316
Johnson High School    69.469286  67.990220  68.637730  69.287393
Pena High School       71.996364  72.396000  72.523438  71.187845
Rodriguez High School  71.940722  71.779808  72.364811  72.154626
Shelton High School    72.932075  72.506696  70.097087  72.331536
Thomas High School     69.234273  70.057007  69.657831  69.369822
Wilson High School     69.212361  69.455446  68.378965  69.787472
Wright High School     71.741176  72.179012  73.275862  70.848238

## Reading Score by Year

In [15]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby(["school_name"])["reading_score"].mean()
year_ten_scores = year_ten.groupby(["school_name"])["reading_score"].mean()
year_eleven_scores = year_eleven.groupby(["school_name"])["reading_score"].mean()
year_twelve_scores = year_twelve.groupby(["school_name"])["reading_score"].mean()

# Combine series into single DataFrame
reading_scores_by_year = pd.DataFrame({"Year 9": year_nine_scores,
                                       "Year 10": year_ten_scores,
                                       "Year 11": year_eleven_scores,
                                       "Year 12": year_twelve_scores})

# Minor data wrangling
reading_scores_by_year.index.name = None

# Display the DataFrame
reading_scores_by_year

Year 9    Year 10    Year 11    Year 12
Bailey High School     70.901920  70.848265  70.317346  72.195525
Cabrera High School    71.172348  71.328326  71.201245  71.856021
Figueroa High School   70.261682  67.677588  69.152327  69.082126
Ford High School       69.615846  68.988701  70.735964  68.849722
Griffin High School    72.026895  70.746305  72.385042  69.434932
Hernandez High School  68.477569  70.621842  68.418199  69.244136
Holden High School     71.598425  71.096491  73.310680  70.481928
Huang High School      68.670616  69.516297  68.740638  68.671795
Johnson High School    68.719286  69.295029  69.969115  67.992521
Pena High School       70.949091  72.324000  71.703125  71.513812
Rodriguez High School  70.902921  70.137500  71.424453  71.414449
Shelton High School    70.715094  69.879464  71.150485  69.070081
Thomas High School     69.672451  69.741093  70.749398  68.730769
Wilson High School     68.683043  68.412541  68.796327  69.888143
Wright High School     71.823529  71.386831  70.475862  69.821138

## Scores by School Spending

In [16]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

In [17]:
# Create a copy of the school summary since it has the "Per Student Budget"
#  This step can be skipped but its best to make a copy.
school_spending_df = per_school_summary

In [18]:
# Categorise spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins=spending_bins, labels=group_names)
school_spending_df

School Type  Total Students Total School Budget  \
school_name                                                              
Bailey High School      Government            4976       $3,124,928.00   
Cabrera High School    Independent            1858       $1,081,356.00   
Figueroa High School    Government            2949       $1,884,411.00   
Ford High School        Government            2739       $1,763,916.00   
Griffin High School    Independent            1468         $917,500.00   
Hernandez High School   Government            4635       $3,022,020.00   
Holden High School     Independent             427         $248,087.00   
Huang High School       Government            2917       $1,910,635.00   
Johnson High School     Government            4761       $3,094,650.00   
Pena High School       Independent             962         $585,858.00   
Rodriguez High School   Government            3999       $2,547,363.00   
Shelton High School    Independent            1761       $1,056,600.00   
Thomas High School     Independent            1635       $1,043,130.00   
Wilson High School     Independent            2283       $1,319,574.00   
Wright High School     Independent            1800       $1,049,400.00   

                       Per Student Budget  Average Maths Score  \
school_name                                                      
Bailey High School                  628.0            72.352894   
Cabrera High School                 582.0            71.657158   
Figueroa High School                639.0            68.698542   
Ford High School                    644.0            69.091274   
Griffin High School                 625.0            71.788147   
Hernandez High School               652.0            68.874865   
Holden High School                  581.0            72.583138   
Huang High School                   655.0            68.935207   
Johnson High School                 650.0            68.843100   
Pena High School                    609.0            72.088358   
Rodriguez High School               637.0            72.047762   
Shelton High School                 600.0            72.034072   
Thomas High School                  638.0            69.581651   
Wilson High School                  578.0            69.170828   
Wright High School                  583.0            72.047222   

                       Average Reading Score  % Passing Maths  \
school_name                                                     
Bailey High School                 71.008842        91.639871   
Cabrera High School                71.359526        90.850377   
Figueroa High School               69.077993        81.654798   
Ford High School                   69.572472        82.438846   
Griffin High School                71.245232        91.212534   
Hernandez High School              69.186408        80.949299   
Holden High School                 71.660422        89.929742   
Huang High School                  68.910525        81.693521   
Johnson High School                69.039277        82.062592   
Pena High School                   71.613306        91.683992   
Rodriguez High School              70.935984        90.797699   
Shelton High School                70.257808        91.538898   
Thomas High School                 69.768807        83.853211   
Wilson High School                 68.876916        82.785808   
Wright High School                 70.969444        91.777778   

                       % Passing Reading  % Overall Passing  \
school_name                                                   
Bailey High School             87.379421          80.084405   
Cabrera High School            89.074273          80.785791   
Figueroa High School           82.807731          67.650051   
Ford High School               82.219788          67.469880   
Griffin High School            88.487738          81.335150   
Hernandez High School          81.877023          66.364617   
Holden High School             

In [19]:
#  Calculate averages for the desired columns.
spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_maths = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

/var/folders/11/qyrcbd7x6zn52rpt4mkkv4700000gn/T/ipykernel_43740/1079946528.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
/var/folders/11/qyrcbd7x6zn52rpt4mkkv4700000gn/T/ipykernel_43740/1079946528.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
/var/folders/11/qyrcbd7x6zn52rpt4mkkv4700000gn/T/ipykernel_43740/1079946528.py:4: FutureWarning: The default of observed=Fal

In [20]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores,
                                 "Average Reading Score": spending_reading_scores,
                                 "% Passing Maths": spending_passing_maths,
                                 "% Passing Reading": spending_passing_reading,
                                 "% Overall Passing": overall_passing_spending})

# Formatting
spending_summary["Average Maths Score"] = spending_summary["Average Maths Score"].map("{:,.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:,.2f}".format)
spending_summary["% Passing Maths"] = spending_summary["% Passing Maths"].map("{:,.2f}".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:,.2f}".format)
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"].map("{:,.2f}".format)

# Display results
spending_summary


Average Maths Score Average Reading Score  \
Spending Ranges (Per Student)                                             
<$585                                       71.36                 70.72   
$585-630                                    72.07                 71.03   
$630-645                                    69.85                 69.84   
$645-680                                    68.88                 69.05   

                              % Passing Maths % Passing Reading  \
Spending Ranges (Per Student)                                     
<$585                                   88.84             86.39   
$585-630                                91.52             87.29   
$630-645                                84.69             83.76   
$645-680                                81.57             81.77   

                              % Overall Passing  
Spending Ranges (Per Student)                    
<$585                                     76.72  
$585-630                                  79.88  
$630-645                                  71.00  
$645-680                                  66.76

## Scores by School Size

In [21]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary


School Type  Total Students Total School Budget  \
school_name                                                              
Bailey High School      Government            4976       $3,124,928.00   
Cabrera High School    Independent            1858       $1,081,356.00   
Figueroa High School    Government            2949       $1,884,411.00   
Ford High School        Government            2739       $1,763,916.00   
Griffin High School    Independent            1468         $917,500.00   
Hernandez High School   Government            4635       $3,022,020.00   
Holden High School     Independent             427         $248,087.00   
Huang High School       Government            2917       $1,910,635.00   
Johnson High School     Government            4761       $3,094,650.00   
Pena High School       Independent             962         $585,858.00   
Rodriguez High School   Government            3999       $2,547,363.00   
Shelton High School    Independent            1761       $1,056,600.00   
Thomas High School     Independent            1635       $1,043,130.00   
Wilson High School     Independent            2283       $1,319,574.00   
Wright High School     Independent            1800       $1,049,400.00   

                       Per Student Budget  Average Maths Score  \
school_name                                                      
Bailey High School                  628.0            72.352894   
Cabrera High School                 582.0            71.657158   
Figueroa High School                639.0            68.698542   
Ford High School                    644.0            69.091274   
Griffin High School                 625.0            71.788147   
Hernandez High School               652.0            68.874865   
Holden High School                  581.0            72.583138   
Huang High School                   655.0            68.935207   
Johnson High School                 650.0            68.843100   
Pena High School                    609.0            72.088358   
Rodriguez High School               637.0            72.047762   
Shelton High School                 600.0            72.034072   
Thomas High School                  638.0            69.581651   
Wilson High School                  578.0            69.170828   
Wright High School                  583.0            72.047222   

                       Average Reading Score  % Passing Maths  \
school_name                                                     
Bailey High School                 71.008842        91.639871   
Cabrera High School                71.359526        90.850377   
Figueroa High School               69.077993        81.654798   
Ford High School                   69.572472        82.438846   
Griffin High School                71.245232        91.212534   
Hernandez High School              69.186408        80.949299   
Holden High School                 71.660422        89.929742   
Huang High School                  68.910525        81.693521   
Johnson High School                69.039277        82.062592   
Pena High School                   71.613306        91.683992   
Rodriguez High School              70.935984        90.797699   
Shelton High School                70.257808        91.538898   
Thomas High School                 69.768807        83.853211   
Wilson High School                 68.876916        82.785808   
Wright High School                 70.969444        91.777778   

                       % Passing Reading  % Overall Passing  \
school_name                                                   
Bailey High School             87.379421          80.084405   
Cabrera High School            89.074273          80.785791   
Figueroa High School           82.807731          67.650051   
Ford High School               82.219788          67.469880   
Griffin High School            88.487738          81.335150   
Hernandez High School          81.877023          66.364617   
Holden High School             

In [22]:
# Categorize the spending based on the bins
per_school_summary["School Size"] = pd.cut(school_spending_df["Total Students"], bins=size_bins, labels=group_names)
per_school_summary


School Type  Total Students Total School Budget  \
school_name                                                              
Bailey High School      Government            4976       $3,124,928.00   
Cabrera High School    Independent            1858       $1,081,356.00   
Figueroa High School    Government            2949       $1,884,411.00   
Ford High School        Government            2739       $1,763,916.00   
Griffin High School    Independent            1468         $917,500.00   
Hernandez High School   Government            4635       $3,022,020.00   
Holden High School     Independent             427         $248,087.00   
Huang High School       Government            2917       $1,910,635.00   
Johnson High School     Government            4761       $3,094,650.00   
Pena High School       Independent             962         $585,858.00   
Rodriguez High School   Government            3999       $2,547,363.00   
Shelton High School    Independent            1761       $1,056,600.00   
Thomas High School     Independent            1635       $1,043,130.00   
Wilson High School     Independent            2283       $1,319,574.00   
Wright High School     Independent            1800       $1,049,400.00   

                       Per Student Budget  Average Maths Score  \
school_name                                                      
Bailey High School                  628.0            72.352894   
Cabrera High School                 582.0            71.657158   
Figueroa High School                639.0            68.698542   
Ford High School                    644.0            69.091274   
Griffin High School                 625.0            71.788147   
Hernandez High School               652.0            68.874865   
Holden High School                  581.0            72.583138   
Huang High School                   655.0            68.935207   
Johnson High School                 650.0            68.843100   
Pena High School                    609.0            72.088358   
Rodriguez High School               637.0            72.047762   
Shelton High School                 600.0            72.034072   
Thomas High School                  638.0            69.581651   
Wilson High School                  578.0            69.170828   
Wright High School                  583.0            72.047222   

                       Average Reading Score  % Passing Maths  \
school_name                                                     
Bailey High School                 71.008842        91.639871   
Cabrera High School                71.359526        90.850377   
Figueroa High School               69.077993        81.654798   
Ford High School                   69.572472        82.438846   
Griffin High School                71.245232        91.212534   
Hernandez High School              69.186408        80.949299   
Holden High School                 71.660422        89.929742   
Huang High School                  68.910525        81.693521   
Johnson High School                69.039277        82.062592   
Pena High School                   71.613306        91.683992   
Rodriguez High School              70.935984        90.797699   
Shelton High School                70.257808        91.538898   
Thomas High School                 69.768807        83.853211   
Wilson High School                 68.876916        82.785808   
Wright High School                 70.969444        91.777778   

                       % Passing Reading  % Overall Passing  \
school_name                                                   
Bailey High School             87.379421          80.084405   
Cabrera High School            89.074273          80.785791   
Figueroa High School           82.807731          67.650051   
Ford High School               82.219788          67.469880   
Griffin High School            88.487738          81.335150   
Hernandez High School          81.877023          66.364617   
Holden High School             

In [23]:
# Calculate averages for the desired columns.
size_maths_scores = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_maths = per_school_summary.groupby(["School Size"])["% Passing Maths"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()


/var/folders/11/qyrcbd7x6zn52rpt4mkkv4700000gn/T/ipykernel_43740/241835436.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_maths_scores = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
/var/folders/11/qyrcbd7x6zn52rpt4mkkv4700000gn/T/ipykernel_43740/241835436.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
/var/folders/11/qyrcbd7x6zn52rpt4mkkv4700000gn/T/ipykernel_43740/241835436.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in

In [24]:
# Assemble into DataFrame
size_summary = pd.DataFrame({"Average Maths Score": size_maths_scores,
                                 "Average Reading Score": size_reading_scores,
                                 "% Passing Maths": size_passing_maths,
                                 "% Passing Reading": size_passing_reading,
                                 "% Overall Passing": size_overall_passing})

# Display results
size_summary


Average Maths Score  Average Reading Score  \
School Size                                                      
Small (<1000)                 72.335748              71.636864   
Medium (1000-2000)            71.421650              70.720164   
Large (2000-5000)             69.751809              69.576052   

                    % Passing Maths  % Passing Reading  % Overall Passing  
School Size                                                                
Small (<1000)             90.806867          87.557513          79.066348  
Medium (1000-2000)        89.846560          86.714149          78.039785  
Large (2000-5000)         84.252804          83.301185          70.293507

## Scores by School Type

In [25]:
# Create new series using groupby for:
# Type | Average Maths Score | Average Reading Score | % Passing Maths | % Passing Reading | % Overall Passing

type_maths_scores = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_maths = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
type_overall_passing = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()


In [26]:
# Assemble into DataFrame
type_summary = pd.DataFrame({"Average Maths Score": type_maths_scores,
                                 "Average Reading Score": type_reading_scores,
                                 "% Passing Maths": type_passing_maths,
                                 "% Passing Reading": type_passing_reading,
                                 "% Overall Passing": type_overall_passing})

# Display results
type_summary


Average Maths Score  Average Reading Score  % Passing Maths  \
School Type                                                                
Government             69.834806              69.675929        84.462375   
Independent            71.368822              70.718933        89.204043   

             % Passing Reading  % Overall Passing  
School Type                                        
Government           83.587562          70.698993  
Independent          86.247789          76.973340